<a href="https://colab.research.google.com/github/Leia2000/Bachelorarbeit_Skripte/blob/main/create_and_export_prediction_map_from_SAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rasterio
!pip install numpy
!pip install os

In [ ]:
# Install the required libraries
#SAM
!pip install git+https://github.com/facebookresearch/segment-anything.git
#Transformers
!pip install -q git+https://github.com/huggingface/transformers.git
#Datasets to prepare data and monai if you want to use special loss functions
!pip install datasets
!pip install -q monai
#Patchify to divide large images into smaller patches for training. (Not necessary for smaller images)
!pip install patchify

In [ ]:
import rasterio
import numpy as np
import os
import matplotlib.pyplot as plt
from datasets import Dataset
from PIL import Image
import tifffile
from patchify import patchify  #Only to handle large images
import random
from scipy import ndimage
from transformers import SamModel, SamConfig, SamProcessor
import torch

In [ ]:
# Directory containing TIF images
dir_path = '/content/drive/MyDrive/Bachelorarbeit_Hannes_Grünbeck/TIF/Quadrate'

# List all TIF files in the directory
tif_files = [os.path.join(dir_path, f) for f in os.listdir(dir_path) if f.endswith('.tif')]

# Initialize an empty list to hold the arrays
image_stack = []
metadata_list = []
# Total number of files
total_files = len(tif_files)
# Loop through each file, read the data, and stack

for idx, tif_file in enumerate(tif_files, start=1):
    print(f"Processing file {idx}/{total_files}: {os.path.basename(tif_file)}")
    with rasterio.open(tif_file) as src:
        data = src.read(1)  # Read the first band
        image_stack.append(data)
        metadata_list.append(src.meta)  # Save metadata

# Convert list to numpy array (stacked)
#stacked_array = np.stack(image_stack, axis=0)

In [ ]:
# Directory containing TIF images
dir_path = '/content/drive/MyDrive/Bachelorarbeit_Hannes_Grünbeck/TIF/Quadrate'

# List all TIF files in the directory
tif_files = [os.path.join(dir_path, f) for f in os.listdir(dir_path) if f.endswith('.tif')]

# Initialize an empty list to hold the arrays
image_stack = []
metadata_list = []
# Total number of files
total_files = len(tif_files)

# Target shape (lowest shape)
target_shape = (6538, 6754)

# Loop through each file, read the data, and stack
for idx, tif_file in enumerate(tif_files, start=1):
    print(f"Processing file {idx}/{total_files}: {os.path.basename(tif_file)}")
    with rasterio.open(tif_file) as src:
        data = src.read(1)  # Read the first band

        # Cut the image to the target shape
        data = data[:target_shape[1], :target_shape[0]]

        image_stack.append(data)
        metadata_list.append(src.meta)  # Save metadata

# Convert list to numpy array (stacked)
stacked_array = np.stack(image_stack, axis=0)

Processing file 1/4: oben_links.tif
Processing file 2/4: unten_links.tif
Processing file 3/4: unten_rechts.tif
Processing file 4/4: oben_rechts.tif


In [ ]:
# Check the dimensions of each image in the stack
for i, image in enumerate(image_stack):
    height, width = image.shape  # Assuming the images are 2D
    print(f"Image {i + 1}: Width = {width}, Height = {height}")

Image 1: Width = 6538, Height = 6754
Image 2: Width = 6538, Height = 6754
Image 3: Width = 6538, Height = 6754
Image 4: Width = 6538, Height = 6754


In [ ]:
stacked_array = np.stack(image_stack, axis=0)

In [ ]:
#Get bounding boxes from mask.
def get_bounding_box(ground_truth_map):
  # get bounding box from mask
  y_indices, x_indices = np.where(ground_truth_map > 0)
  x_min, x_max = np.min(x_indices), np.max(x_indices)
  y_min, y_max = np.min(y_indices), np.max(y_indices)
  # add perturbation to bounding box coordinates
  H, W = ground_truth_map.shape
  x_min = max(0, x_min - np.random.randint(0, 20))
  x_max = min(W, x_max + np.random.randint(0, 20))
  y_min = max(0, y_min - np.random.randint(0, 20))
  y_max = min(H, y_max + np.random.randint(0, 20))
  bbox = [x_min, y_min, x_max, y_max]

  return bbox

In [ ]:
# Load the model configuration
model_config = SamConfig.from_pretrained("facebook/sam-vit-base")
processor = SamProcessor.from_pretrained("facebook/sam-vit-base")

# Create an instance of the model architecture with the loaded configuration
my_mito_model = SamModel(config=model_config)
#Update the model by loading the weights from saved file.
my_mito_model.load_state_dict(torch.load("/content/drive/MyDrive/Bachelorarbeit_Hannes_Grünbeck/Ergebnisse_SAM/SAM_Twigs.pth"))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/6.57k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

<ipython-input-8-f46dcd68edac>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  my_mito_model.load_state_dict(torch.load("/content/drive/MyDrive/Bachelorarbeit_Hannes_Grünb

<All keys matched successfully>

In [ ]:
# set the device to cuda if available, otherwise use cpu
device = "cuda" if torch.cuda.is_available() else "cpu"
my_mito_model.to(device)

In [ ]:
#Apply a trained model on large image
#large_test_images = tifffile.imread("/content/drive/MyDrive/Bachelorarbeit_Hannes_Grünbeck/TIF/ortho_plot_8_230726_modified_georef.tif")
large_test_images = stacked_array
large_test_image = large_test_images[0]
patches = patchify(large_test_image, (256, 256), step=256)  #Step=256 for 256 patches means no overlap

In [ ]:
"""
input_points (torch.FloatTensor of shape (batch_size, num_points, 2)) —
Input 2D spatial points, this is used by the prompt encoder to encode the prompt.
Generally yields to much better results. The points can be obtained by passing a
list of list of list to the processor that will create corresponding torch tensors
of dimension 4. The first dimension is the image batch size, the second dimension
is the point batch size (i.e. how many segmentation masks do we want the model to
predict per input point), the third dimension is the number of points per segmentation
mask (it is possible to pass multiple points for a single mask), and the last dimension
is the x (vertical) and y (horizontal) coordinates of the point. If a different number
of points is passed either for each image, or for each mask, the processor will create
“PAD” points that will correspond to the (0, 0) coordinate, and the computation of the
embedding will be skipped for these points using the labels.

"""
# Define the size of your array
array_size = 256

# Define the size of your grid
grid_size = 10

# Generate the grid points
x = np.linspace(0, array_size-1, grid_size)
y = np.linspace(0, array_size-1, grid_size)

# Generate a grid of coordinates
xv, yv = np.meshgrid(x, y)

# Convert the numpy arrays to lists
xv_list = xv.tolist()
yv_list = yv.tolist()

# Combine the x and y coordinates into a list of list of lists
input_points = [[[int(x), int(y)] for x, y in zip(x_row, y_row)] for x_row, y_row in zip(xv_list, yv_list)]

#We need to reshape our nxn grid to the expected shape of the input_points tensor
# (batch_size, point_batch_size, num_points_per_image, 2),
# where the last dimension of 2 represents the x and y coordinates of each point.
#batch_size: The number of images you're processing at once.
#point_batch_size: The number of point sets you have for each image.
#num_points_per_image: The number of points in each set.
input_points = torch.tensor(input_points).view(1, 1, grid_size*grid_size, 2)

In [ ]:
print(np.array(input_points).shape)

(1, 1, 100, 2)


In [ ]:
patches.shape

(26, 25, 256, 256)

In [ ]:
# Select a random patch for segmentation

# Compute the total number of 256x256 arrays
num_arrays = patches.shape[0] * patches.shape[1]
# Select a random index
index = np.random.choice(num_arrays)
# Compute the indices in the original array
#i = index // patches.shape[1]
#j = index % patches.shape[1]

#Or pick a specific patch for study.
i, j = 0,0

# Selectelected patch for segmentation
random_array = patches[i, j]


single_patch = Image.fromarray(random_array)

In [ ]:
# prepare image for the model

#First try without providing any prompt (no bounding box or input_points)
#Convert to RGB for the correct number dimensions
#inputs = processor(single_patch.convert("RGB"), return_tensors="pt")
#Now try with bounding boxes. Remember to uncomment.
#Convert to RGB for the correct number dimensions
inputs = processor(single_patch.convert("RGB"), input_points=input_points, return_tensors="pt") # Convert to RGB before passing input_points
# Move the input tensor to the GPU if it's not already there
inputs = {k: v.to(device) for k, v in inputs.items()}
my_mito_model.eval()
# forward pass
with torch.no_grad():
  outputs = my_mito_model(**inputs, multimask_output=False)
# apply sigmoid
single_patch_prob = torch.sigmoid(outputs.pred_masks.squeeze(1))
# convert soft mask to hard mask
single_patch_prob = single_patch_prob.cpu().numpy().squeeze()
single_patch_prediction = (single_patch_prob > 0.5).astype(np.uint8)

/usr/local/lib/python3.11/dist-packages/transformers/image_processing_utils.py:41: UserWarning: The following named arguments are not valid for `SamImageProcessor.preprocess` and were ignored: 'point_pad_value'
  return self.preprocess(images, **kwargs)


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot the first image on the left
axes[0].imshow(np.array(single_patch), cmap='gray')  # Assuming the first image is grayscale
axes[0].set_title("Image")

# Plot the second image on the right
axes[1].imshow(single_patch_prob)  # Assuming the second image is grayscale
axes[1].set_title("Probability Map")

# Plot the second image on the right
axes[2].imshow(single_patch_prediction, cmap='gray')  # Assuming the second image is grayscale
axes[2].set_title("Mask")

# Hide axis ticks and labels
for ax in axes:
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])

# Display the images side by side
plt.show()

In [ ]:
import rasterio
from rasterio.transform import Affine
import numpy as np
from PIL import Image

###########################################
###  Iterate and export img, mask and   ###
###  prediction for ONE items in stack  ###
###########################################


# Get CRS data from the original image metadata
crs = metadata_list[0]['crs']
transform = metadata_list[0]['transform']
height, width = large_test_image.shape


# Initialize full-size arrays for probability map and prediction mask
prob_map_full = np.zeros_like(large_test_image, dtype=np.float32)  # Use appropriate data type
pred_mask_full = np.zeros_like(large_test_image, dtype=np.uint8)  # Use appropriate data type

# Function to update the full maps with a patch
def update_full_maps(patch_prob, patch_mask, row, col):
    prob_map_full[row * 256:(row + 1) * 256, col * 256:(col + 1) * 256] = patch_prob
    pred_mask_full[row * 256:(row + 1) * 256, col * 256:(col + 1) * 256] = patch_mask

# Iterate through all patches and make predictions
for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):
        random_array = patches[i, j]
        single_patch = Image.fromarray(random_array)

        # Model prediction for the current patch
        inputs = processor(single_patch.convert("RGB"), input_points=input_points, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        my_mito_model.eval()
        with torch.no_grad():
            outputs = my_mito_model(**inputs, multimask_output=False)

        # Process outputs to get probability map and prediction mask for the patch
        single_patch_prob = torch.sigmoid(outputs.pred_masks.squeeze(1))
        single_patch_prob = single_patch_prob.cpu().numpy().squeeze()
        single_patch_prediction = (single_patch_prob > 0.5).astype(np.uint8)

        # Update the full-size maps with the patch results
        update_full_maps(single_patch_prob, single_patch_prediction, i, j)


# Export to Google Drive
drive_path = "/content/drive/MyDrive/Bachelorarbeit_Hannes_Grünbeck/Ergebnisse_SAM/"
"""
# Original image (if needed)
with rasterio.open(drive_path + 'original_image1.tif', 'w', **metadata_list[0]) as dst:
    dst.write(large_test_image, 1)
"""
# Probability map
with rasterio.open(drive_path + 'probability_map_full1.tif', 'w',
                   driver='GTiff', width=width, height=height, count=1,
                   dtype=prob_map_full.dtype, crs=crs, transform=transform) as dst:
    dst.write(prob_map_full, 1)

# Prediction mask
with rasterio.open(drive_path + 'prediction_mask_full1.tif', 'w',
                   driver='GTiff', width=width, height=height, count=1,
                   dtype=pred_mask_full.dtype, crs=crs, transform=transform) as dst:
    dst.write(pred_mask_full, 1)

In [ ]:
import rasterio
from rasterio.transform import Affine
import numpy as np
from PIL import Image

###########################################
###  Iterate and export img, mask and   ###
###  prediction for ALL items in stack  ###
###########################################

# Function to update the full maps with a patch
def update_full_maps(patch_prob, patch_mask, row, col):
    prob_map_full[row * 256:(row + 1) * 256, col * 256:(col + 1) * 256] = patch_prob
    pred_mask_full[row * 256:(row + 1) * 256, col * 256:(col + 1) * 256] = patch_mask

# Process each image in the stack
for image_index in range(large_test_images.shape[0]):
    large_test_image = large_test_images[image_index]

    # Get CRS and transform data from the metadata of the CURRENT image
    crs = metadata_list[image_index]['crs']
    transform = metadata_list[image_index]['transform']
    height, width = large_test_image.shape

    # Patchify the current image
    patches = patchify(large_test_image, (256, 256), step=256)

    # Initialize full-size arrays for probability map and prediction mask
    prob_map_full = np.zeros_like(large_test_image, dtype=np.float32)
    pred_mask_full = np.zeros_like(large_test_image, dtype=np.uint8)

    # Iterate through all patches and make predictions
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            random_array = patches[i, j]
            single_patch = Image.fromarray(random_array)

            # Model prediction for the current patch (same as before)
            #inputs = processor(single_patch.convert("RGB"), input_points=input_points, return_tensors="pt")
            inputs = processor(single_patch.convert("RGB"), return_tensors="pt")
            inputs = {k: v.to(device) for k, v in inputs.items()}
            my_mito_model.eval()
            with torch.no_grad():
                outputs = my_mito_model(**inputs, multimask_output=False)

            # Process outputs (same as before)
            single_patch_prob = torch.sigmoid(outputs.pred_masks.squeeze(1))
            single_patch_prob = single_patch_prob.cpu().numpy().squeeze()
            single_patch_prediction = (single_patch_prob > 0.5).astype(np.uint8)

            # Update the full-size maps with the patch results
            update_full_maps(single_patch_prob, single_patch_prediction, i, j)

    # Export to Google Drive, including image_index in filenames
    drive_path = "/content/drive/MyDrive/Bachelorarbeit_Hannes_Grünbeck/Ergebnisse_SAM/"

    with rasterio.open('/content/drive/MyDrive/Bachelorarbeit_Hannes_Grünbeck/Ergebnisse_SAM/original_img/' + f'original_image_{image_index + 1}.tif', 'w', **metadata_list[image_index]) as dst:
        dst.write(large_test_image, 1)  # Writing only the first band

    with rasterio.open('/content/drive/MyDrive/Bachelorarbeit_Hannes_Grünbeck/Ergebnisse_SAM/probability/' + f'probability_{image_index + 1}.tif', 'w',
                      driver='GTiff', width=width, height=height, count=1,
                      dtype=prob_map_full.dtype, crs=crs, transform=transform) as dst:
        dst.write(prob_map_full, 1)

    with rasterio.open('/content/drive/MyDrive/Bachelorarbeit_Hannes_Grünbeck/Ergebnisse_SAM/mask/' + f'mask_{image_index + 1}.tif', 'w',
                      driver='GTiff', width=width, height=height, count=1,
                      dtype=pred_mask_full.dtype, crs=crs, transform=transform) as dst:
        dst.write(pred_mask_full, 1)